## Installing the hooks
For installation, one needs to copy the hooks files from the 'hooks/' folder to the '.git/hooks/' folder. This can be done by executing the 'install-hooks.sh' Bash script.

Open Git Bash, and type in:

```bash
bash install-hooks.sh
```

To verify the installation has been successful, navigate to the '.git/hooks/' folder and check for our hook files, for example if the post-commit file is present.